## Deliverable 2. Create a Customer Travel Destinations Map.

In [36]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
%matplotlib inline
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [12]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,80.49,74,2,17.31,clear sky
1,1,Egvekinot,RU,66.3167,-179.1667,20.46,77,31,3.27,scattered clouds
2,2,Lasa,IT,46.6166,10.7002,37.40,70,0,2.30,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,44.60,57,20,8.05,few clouds
4,4,Sola,VU,-13.8833,167.5500,83.16,75,35,15.41,scattered clouds


In [13]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,80.49,74,2,17.31,clear sky
4,4,Sola,VU,-13.8833,167.5500,83.16,75,35,15.41,scattered clouds
7,7,Cabo San Lucas,MX,22.8909,-109.9124,81.00,31,1,11.50,clear sky
8,8,Grand Gaube,MU,-20.0064,57.6608,75.99,85,98,5.99,overcast clouds
11,11,Rikitea,PF,-23.1203,-134.9692,77.76,72,33,4.74,scattered clouds
12,12,Mahebourg,MU,-20.4081,57.7000,78.80,83,75,10.36,light rain
13,13,Dong Hoi,VN,17.4833,106.6000,76.77,88,0,6.17,clear sky
18,18,San Pedro,PH,14.3500,121.0167,82.40,83,75,3.44,broken clouds
30,30,Hithadhoo,MV,-0.6000,73.0833,82.08,68,60,6.58,broken clouds
31,31,High Rock,BS,26.6208,-78.2833,80.60,83,75,13.80,broken clouds


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                205
City                   205
Country                203
Lat                    205
Lng                    205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

In [24]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                203
City                   203
Country                203
Lat                    203
Lng                    203
Max Temp               203
Humidity               203
Cloudiness             203
Wind Speed             203
Current Description    203
dtype: int64

In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,80.49,clear sky,-9.8000,-139.0333,
4,Sola,VU,83.16,scattered clouds,-13.8833,167.5500,
7,Cabo San Lucas,MX,81.00,clear sky,22.8909,-109.9124,
8,Grand Gaube,MU,75.99,overcast clouds,-20.0064,57.6608,
11,Rikitea,PF,77.76,scattered clouds,-23.1203,-134.9692,
12,Mahebourg,MU,78.80,light rain,-20.4081,57.7000,
13,Dong Hoi,VN,76.77,clear sky,17.4833,106.6000,
18,San Pedro,PH,82.40,broken clouds,14.3500,121.0167,
30,Hithadhoo,MV,82.08,broken clouds,-0.6000,73.0833,
31,High Rock,BS,80.60,broken clouds,26.6208,-78.2833,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()     
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [45]:
# 7. Drop the rows where there is no Hotel Name.
# replacing empty values in Hotel Name column with NaN
# used code from https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)

In [46]:
#dropping rows which contain NaN
hotel_df.dropna(subset=["Hotel Name"], inplace = True)

In [48]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,80.49,clear sky,-9.8000,-139.0333,Villa Enata
4,Sola,VU,83.16,scattered clouds,-13.8833,167.5500,Leumerus Bungalows
7,Cabo San Lucas,MX,81.00,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
8,Grand Gaube,MU,75.99,overcast clouds,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
11,Rikitea,PF,77.76,scattered clouds,-23.1203,-134.9692,Pension Maro'i


In [52]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [53]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [56]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))